In [1]:
import numpy as np

In [9]:
def read_data_file_freq(file_path):
    headers = []
    data_blocks = []

    with open(file_path, 'r') as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line:
            header = float(line)
            headers.append(header)

            i += 1
            block_lines = []
            while i < len(lines) and lines[i].strip():
                aux = list(map(float, lines[i].split()))
                block_lines.append(aux)
                i += 1
            data_block = np.array(block_lines)
            data_blocks.append(data_block)
        i += 1

    headers_array = np.array(headers)
    data_blocks_array = np.array(data_blocks)

    # now we dupplicate the

    return headers_array, data_blocks_array


In [10]:
headers_freq, data_blocks_freq = read_data_file_freq("../data/solution.txt")

In [12]:
headers_freq[200:]

array([ 100. ,  100.5,  101. , ..., 1199. , 1199.5, 1200. ])

In [13]:
data_blocks_freq[200:]

array([[[-0.534272  , -1.14268   , -1.44007   , ...,  2.65007   ,
          1.5049    ,  0.375021  ],
        [-1.74827   , -2.36841   , -2.66905   , ...,  1.52732   ,
          0.345551  , -0.816509  ],
        [-2.76284   , -3.37624   , -3.66584   , ...,  0.571735  ,
         -0.647945  , -1.82822   ],
        ...,
        [ 1.4667    ,  1.07457   ,  0.953115  , ...,  4.25834   ,
          3.11288   ,  2.1496    ],
        [ 1.39898   ,  0.907671  ,  0.701526  , ...,  4.3312    ,
          3.20659   ,  2.17983   ],
        [ 0.631689  ,  0.064852  , -0.20293   , ...,  3.69791   ,
          2.57493   ,  1.49114   ]],

       [[-0.838373  , -1.60497   , -2.03409   , ...,  2.70009   ,
          1.49578   ,  0.23432   ],
        [-2.25221   , -3.0411    , -3.47432   , ...,  1.51691   ,
          0.209834  , -1.13061   ],
        [-3.46901   , -4.24657   , -4.66017   , ...,  0.437068  ,
         -0.955174  , -2.33832   ],
        ...,
        [ 1.45578   ,  1.0627    ,  0.915315  , ...,  

In [49]:
# Error analysis Backward Euler
T = 1.0
t = 0.0
dt = 0.1
numDt = 8
n = 8
nx = n
ny = n
X = np.linspace(0, 2*np.pi, nx, endpoint=False)
Y = np.linspace(0, 2*np.pi, ny, endpoint=False)
X, Y = np.meshgrid(X, Y)

# wave numbers
kx = np.fft.fftfreq(nx) * nx
ky = np.fft.fftfreq(ny) * ny
kx, ky = np.meshgrid(kx, ky)

aux = kx**2 + ky**2

# real solution of u_t + Δu + Δ^2u +0.5 * |∇u|^2 = 0
u = np.sin(X) + np.sin(Y) +np.sin(X+Y) +np.sin(T)
uhat = np.fft.fft2(u)

In [50]:
def print_my(u0hat):
  for i in range(nx):
      for j in range(ny):
          if (np.abs(u0hat[i, j]) > 1e-12):
              print(i, j, u0hat[i, j])
  print("\n")

In [57]:
# Euler
M = np.zeros(numDt-1)
dt = 0.1
for i in range(1, numDt):  # Change the range to start from 1
    # initial condition
    print(i)
    u0 = np.sin(X) + np.sin(Y) + np.sin(X+Y)
    u0hat = np.fft.fft2(u0)
    t = 0.0
    # linear operator
    L = 1.0/(1 + dt*aux * (aux - 1))
    while (t < T-1e-12):
        # g =((2*sin(t) + 2*sin(x) + 2*sin(y) + 4)*sin(x + y))/2 - cos(x + y)^2/2 + ((2*sin(x) + 2*sin(y))*sin(t))/2 - cos(t)^2/2 - cos(x)^2/2 - cos(y)^2/2 + sin(x)*sin(y) + cos(t) + 2
        g = ((2*np.sin(t) + 2*np.sin(X) + 2*np.sin(Y) + 4)*np.sin(X + Y))/2 - np.cos(X + Y)**2/2 + ((2*np.sin(X) + 2*np.sin(Y))*np.sin(t))/2 - np.cos(t)**2/2 - np.cos(X)**2/2 - np.cos(Y)**2/2 + np.sin(X)*np.sin(Y) + np.cos(t) + 2
        ghat = np.fft.fft2(g)
        # compute the fft of the nonlinear term 0.5 * |∇u|^2
        # print("u0hat")
        # print_my(u0hat)
        # print("Fu_diff")
        # print_my(1j*kx*u0hat)
        u1 = np.fft.ifft2(u0hat)
        u2 = -0.5*u1**2
        u2hat = np.fft.fft2(u2)
        # ux = np.fft.ifft2(1j*kx*u0hat)
        # uy = np.fft.ifft2(1j*ky*u0hat)
        # print("ux^2")
        # print((ux**2).real)
        # grad_sq = np.fft.fft2(-0.5*(ux**2 + uy**2))
        # print("print the whole thing:")
        # print_my(grad_sq)
        # print("u0hat")
        # print_my(u0hat)
        # print("grad_sq")
        # print_my(grad_sq)
        # print("ghat")
        # print_my(ghat)
        u0hat = L*(u0hat + dt*u2hat + dt*ghat)
        # print("u0hat")
        # print_my(u0hat)
        u0 = np.fft.ifft2(u0hat)
        # print("u0")
        # print_my(u0.real)
        u0=u0.flatten()
        # print(np.linalg.norm(u0, ord=2))
        t += dt
    u0 = np.fft.ifft2(u0hat)
    u0 = u0-u
    u0 = u0.flatten()
    M[i-1] = np.linalg.norm(u0, ord=2)
    dt = dt/2
print(M)

1
2
3
4
5
6
7
[8.85629209 8.90834015 8.93481098 8.94818195 8.95490462 8.95827568
 8.9599637 ]


In [11]:
# BFD-2
M = np.zeros(numDt-1)
dt = 0.1
for i in range(1, numDt):  # Change the range to start from 1
    # initial condition
    print(i)
    u0 = np.sin(2*X)*np.sin(2*Y)
    u0hat = np.fft.fft2(u0)
    t = 0.0
    u1hat = u0hat
    Le = 1.0/(1 + dt*aux)
    Lb = 1.0/(3 + 2*dt*aux)
    while (t < T-1e-12):
        if t < 1e-10:# we do one step of Euler
            u1hat = Le*u0hat
        else: # we do BFD-2
            uauxhat = Lb*(4*u1hat - u0hat)
            u0hat = u1hat
            u1hat = uauxhat
        t += dt
    u1 = np.fft.ifft2(u1hat)
    u1 = u1-u
    u1 = u1.flatten()
    M[i-1] = np.linalg.norm(u1,ord=2)
    dt = dt/2
print(M)

1
2
3
4
5
6
7
8
9
[3.72617937e-03 5.59703515e-04 1.19884148e-04 2.77832527e-05
 6.68440179e-06 1.63906518e-06 4.05799273e-07 1.00956203e-07
 2.51774884e-08]


In [56]:
for i in range(M.size-1):
    tmp = np.log(M[i]/M[i+1])/np.log(2)
    print(tmp)

-0.00845384254700725
-0.0042805615186909144
-0.0021573840183279143
-0.0010834735639300686
-0.0005429986455944193
-0.0002718233193750345


In [22]:
# Euler
u0 = np.sin(2*X)*np.sin(2*Y)
u0hat = np.fft.fft2(u0)
t = 0.0
dt = 0.025
L = 1.0/(1 + dt*aux)  # linear operator
print_my(u0hat)
while (t < T-1e-12):
    u0hat = L*u0hat
    print_my(u0hat)
    t += dt
u0 = np.fft.ifft2(u0hat)
u0 = u0-u
u0 = u0.flatten()
np.linalg.norm(u0,ord=np.inf)

2 2 (-16+5.878304635907295e-15j)
2 6 (16+0j)
6 2 (16+0j)
6 6 (-16-5.878304635907295e-15j)


2 2 (-13.333333333333334+4.898587196589413e-15j)
2 6 (13.333333333333334+0j)
6 2 (13.333333333333334+0j)
6 6 (-13.333333333333334-4.898587196589413e-15j)


2 2 (-11.111111111111112+4.0821559971578446e-15j)
2 6 (11.111111111111112+0j)
6 2 (11.111111111111112+0j)
6 6 (-11.111111111111112-4.0821559971578446e-15j)


2 2 (-9.259259259259261+3.401796664298204e-15j)
2 6 (9.259259259259261+0j)
6 2 (9.259259259259261+0j)
6 6 (-9.259259259259261-3.401796664298204e-15j)


2 2 (-7.716049382716052+2.834830553581837e-15j)
2 6 (7.716049382716052+0j)
6 2 (7.716049382716052+0j)
6 6 (-7.716049382716052-2.834830553581837e-15j)


2 2 (-6.430041152263376+2.3623587946515308e-15j)
2 6 (6.430041152263376+0j)
6 2 (6.430041152263376+0j)
6 6 (-6.430041152263376-2.3623587946515308e-15j)


2 2 (-5.358367626886147+1.9686323288762757e-15j)
2 6 (5.358367626886147+0j)
6 2 (5.358367626886147+0j)
6 6 (-5.358367626886147-1.9686323

0.0003449152088941221

In [31]:
s=u0-u

In [35]:
s.shape

(64,)

In [34]:
s=s.flatten()

In [23]:
r = np.array([1,2,3,4])
np.linalg.norm(r,ord=2)

5.477225575051661